In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
import predict
import base
import shop_classify as sc

import matplotlib
myfont = matplotlib.font_manager.FontProperties(fname=r'C:/Windows/Fonts/msyh.ttf')
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

%load_ext autoreload
%autoreload 2

C:\Program Files\Anaconda3\lib\site-packages\matplotlib\__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
#读取休假表
calenders = base.ReadCalendarCSV();
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
###读取处理过的商家信息
shop_infos = base.ReadProcessShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();


In [3]:
%reload_ext autoreload

In [4]:
###读取处理过的商家信息
shop_infos = base.ReadProcessShopInfoCSV();

In [5]:
names = ['2_2_1', '2_2_2', '3_3_1', '3_3_2', '3_3_3', 'arima']
predict_predictions = []

for i in range(len(names)):
    predict_predictions.append(pd.read_csv("./data/dataset/dataset/shop_classify/check/predict/" + names[i] + "_prediction.txt", sep='\t', encoding='UTF-8', index_col=0))

In [6]:
#生成训练集

In [7]:
#真实数据2016-9-20~~2016-9-20 + 2016-11-10~~2016-10-24
#calculatePeriodParams(shop_open_dates, user_pay_counts, calenders, start_time, end_time):

In [8]:
predict_pay_counts = pd.read_csv("./data/dataset/dataset/shop_classify/check/predict/preprocess_data.txt", sep="\t", encoding='UTF-8', index_col=0)

In [9]:
predict_pay_counts.columns = range(0,21)

In [10]:
week_num = 3

func_names = ['mean', 'std', 'max', 'min']
funcs = [np.mean, np.std, np.max, np.min]
real_params = {}

for t in range(len(func_names)):
    params = []
    for w in range(week_num):
        params.append(funcs[t](predict_pay_counts[np.array(range(w*7, w*7+6))],axis=1))
    
    real_params[func_names[t]] = params

In [11]:
predict_params = []
weeks = [['0', '1', '2', '3', '4', '5', '6'],['7', '8', '9', '10', '11', '12', '13']]

for i in range(len(names)):
    print(i)
    for w in weeks:
        predction = predict_predictions[i][w];
        predict_params.append([predction.mean(axis=1), predction.std(axis=1), predction.max(axis=1), predction.min(axis=1)])

0
1
2
3
4
5


In [12]:
def initFeatures(t):
    X_train_df = pd.DataFrame()
    X_train_df['mean1'] = real_params['mean'][0]
    X_train_df['mean2'] = real_params['mean'][1]
    X_train_df['mean3'] = real_params['mean'][2]
    X_train_df['std1'] = real_params['std'][0]
    X_train_df['std2'] = real_params['std'][1]
    X_train_df['std3'] = real_params['std'][2]
    X_train_df['min1'] = real_params['min'][0]
    X_train_df['min2'] = real_params['min'][1]
    X_train_df['min3'] = real_params['min'][2]
    X_train_df['max1'] = real_params['max'][0]
    X_train_df['max2'] = real_params['max'][1]
    X_train_df['max3'] = real_params['max'][2]
    X_train_df['mean_diff1'] = real_params['mean'][0] - real_params['mean'][1]
    X_train_df['mean_diff2'] = real_params['mean'][1] - real_params['mean'][2]
    X_train_df['mean_diff3'] = real_params['mean'][2] - real_params['mean'][0]

    X_train_df['std_diff1'] = real_params['std'][0] - real_params['std'][1]
    X_train_df['std_diff2'] = real_params['std'][1] - real_params['std'][2]
    X_train_df['std_diff3'] = real_params['std'][2] - real_params['std'][0]

    X_train_df['max_diff1'] = real_params['max'][0] - real_params['max'][1]
    X_train_df['max_diff2'] = real_params['max'][1] - real_params['max'][2]
    X_train_df['max_diff3'] = real_params['max'][2] - real_params['max'][0]

    X_train_df['min_diff1'] = real_params['min'][0] - real_params['min'][1]
    X_train_df['min_diff2'] = real_params['min'][1] - real_params['min'][2]
    X_train_df['min_diff3'] = real_params['min'][2] - real_params['min'][0]

#     if t == '3_1':
#         X_train_df['mean4_pred_1'] = predict_params[0][0]
#         X_train_df['std4_pred_1'] = predict_params[0][1]
#         X_train_df['max4_pred_1'] = predict_params[0][2]
#         X_train_df['min4_pred_1'] = predict_params[0][3]
#     elif t == '3_2':
#         X_train_df['mean4_pred_1'] = predict_params[1][0]
#         X_train_df['std4_pred_1'] = predict_params[1][1]
#         X_train_df['max4_pred_1'] = predict_params[1][2]
#         X_train_df['min4_pred_1'] = predict_params[1][3]
#     elif t == '3_3':
#         X_train_df['mean4_pred_1'] = predict_params[2][0]
#         X_train_df['std4_pred_1'] = predict_params[2][1]
#         X_train_df['max4_pred_1'] = predict_params[2][2]
#         X_train_df['min4_pred_1'] = predict_params[2][3]


    X_train_df = X_train_df.astype(np.int)
    X_train_df = pd.concat([X_train_df, shop_infos], axis=1)
    return X_train_df;

In [13]:
mean_type = '3_3_3'
X_predict_df = initFeatures(mean_type)

In [14]:
#["2_2_1", '2_2_2', '3_3_1', '3_3_2', '3_3_3', 'arima']
#["2_2_1", '2_2_2', '3_3_1', '3_3_2', '3_3_3', 'arima']
def changeType(name):
    if name == 'arima':
        return 1
    else:
        return 0;

In [15]:
X_predict_df.to_csv("./data/dataset/dataset/shop_classify/check/predict/X_predict.txt", sep='\t', index=True, encoding='UTF-8')

In [120]:
%reload_ext autoreload